In [1]:
import cv2
import numpy as np
from operator import itemgetter, attrgetter 

import win32api
import win32con

### Extracting showing (screen) area

In [2]:
def find_greenScreen(img):
    # set a range of green to detect the green screen
    # [R value, G value, B value]
    lower_green = np.array([0, 130, 0])     
    upper_green = np.array([110, 255, 110])
    
    # fill the selected green area white
    mask = cv2.inRange(img, lower_green, upper_green)
    # reverse mask so the screen area will be filled black
    mask = cv2.bitwise_not(mask)
    # smooth the mask area
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, np.ones((3,3),np.uint8))
    
    # apply mask on the given image
    maskedScreen = cv2.bitwise_and(img, img, mask=mask)
    return mask, maskedScreen

In [10]:
def on_click(event, x, y, p1, p2):
    #the function store the click points
    
    global gloablPts
    if event == cv2.EVENT_LBUTTONDOWN:
        gloablPts.append((x, y))

def draw_tv_screen(screen):
    # copy the original image
    img = screen.copy()                  
    cv2.namedWindow(winname= "TV image")
    # call The click function
    cv2.setMouseCallback("TV image", on_click)
    txt = 'Click on the border of the screen area'
    
    while True:
        cv2.imshow("TV image", img)
        cv2.putText(img, txt, (150, 100), cv2.FONT_HERSHEY_COMPLEX, .6, (0,0,0), 1)
        # change cursor to a crooshair to make drawing easier
        win32api.SetCursor(win32api.LoadCursor(0, win32con.IDC_CROSS))

        # if we have 2 points draw line between them
        if len(gloablPts) == 2:
            cv2.line(img,(gloablPts[-2][0], gloablPts[-2][1]),(gloablPts[-1][0], gloablPts[-1][1]),(0,0,250), 2)

        # if we have more than 2 points draw polygon
        if len(gloablPts) > 2:    
            cv2.fillPoly(img, np.int32([gloablPts]), (0, 0, 250), lineType=cv2.LINE_AA)

        # if we have 4 points close the window and break
        if len(gloablPts) == 4:
            cv2.destroyAllWindows()
            break

        k = cv2.waitKey(30) & 0xFF
        # wait for ESC key or q to exit
        if k == 27 or k == ord('q'):         
            cv2.destroyAllWindows()
            break

### Locate screen area

In [11]:
def get_contours(mask):
    # get image contours
    ret,thresh = cv2.threshold(mask, 127, 255, 0)
    contours,hierarchy = cv2.findContours(thresh, 1, 2)
    # sort contours in descending order
    largest_cnt = sorted(contours, key=cv2.contourArea, reverse = True)
    # screen is the second contour in the list, as the largest contour is the whole image
    largest_cnt = largest_cnt[1]
    
    # find the rectangular that contains the screen
    rect = cv2.minAreaRect(largest_cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    
    # sort points according to their vertical coordinate then to their horizontal coordinate
    sortedPts = sorted(box, key=itemgetter(1, 0))
    # arrange point to be in a clockwise order
    pts = [sortedPts[0], sortedPts[1], sortedPts[3], sortedPts[2]]
    return pts

### 1. Insert photo inside a photo

In [12]:
# define global variable to store points of the selected screen
gloablPts = []

# import input image & TV screen image
inputImage = cv2.imread('autumn.jpg')
screen = cv2.imread('living3.jpg')

# get image shapes
(hScr, wScr) = screen.shape[:2]
(hImg, wImg) = inputImage.shape[:2]

# show screen image to select tv screen 
draw_tv_screen(screen)

# fill screen with black
pts = gloablPts
maskedTV = cv2.fillPoly(screen, np.int32([pts]), (0, 0, 0), lineType=cv2.LINE_AA)


# input points are as same as input image shape
ptImg = np.float32([[0, 0], [wImg, 0], [wImg, hImg], [0, hImg]])
# the output points are as same as tv screen
ptTV = np.float32(pts)

# Calculate transformation matrix
matrix = cv2.getPerspectiveTransform(ptImg, ptTV)

# warp input image to the screen
warpedImg =cv2.warpPerspective(inputImage, matrix, (wScr,hScr))

# mask the wraped image
output = cv2.add(maskedTV, warpedImg)

cv2.imshow('image', output)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
cv2.imwrite('image_inside.jpg', output)

True

### 2. Insert video inside a photo

In [62]:
screen = cv2.imread('green.jpg')
cv2.imshow('image', screen)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
# import tv screen image
screen = cv2.imread('green.jpg')

# get image dimensions
(hTV, wTV) = screen.shape[:2]

# get the masked screen
maskScr, maskedScreen = find_greenScreen(screen)

# find the rectangular that contains the TV screen 
pts = get_contours(maskScr)

# import input video
video = cv2.VideoCapture('video.mp4')

# prepare video parameters for saving
# size is equal to image size not video
size = int(screen.shape[1]),int(screen.shape[0])
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('tv_inside_img.mp4',fourcc, 20, size, True)

# to display video
while video.isOpened():
    ret, frame = video.read()
    
    if ret == True:
        # get frame shape
        (hFrame, wFrame) = frame.shape[:2]

        # get frame border to insert it into the tv screen
        ptFrame = np.float32([[0, 0], [wFrame, 0], [wFrame, hFrame], [0, hFrame]])
        # convert TV screen into float
        ptTV = np.float32(pts)
        
        # Calculate transformation matrix
        matrix = cv2.getPerspectiveTransform(ptFrame, ptTV)
        
        # warp input video to the screen
        warpedFrame =cv2.warpPerspective(frame, matrix, (wTV,hTV))
        
        # create a mask for the input video, which is the revese of the TV screen mask
        maskVideo = cv2.bitwise_not(maskScr)
        
        # mask input video
        maskedVideo = cv2.bitwise_and(warpedFrame, warpedFrame, mask= maskVideo)

        finalFrame = cv2.add(maskedScreen, maskedVideo)
        
        # save video
        out.write(finalFrame)

        cv2.imshow('Video', finalFrame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            cv2.destroyWindow('Video')
    
    else:
        break

out.release()         
cv2.destroyWindow('Video')

### 3. Insert video inside a video

In [22]:
# load screen video & input video
scrVideo = cv2.VideoCapture('TVscreen.mp4')
InputVideo = cv2.VideoCapture('video.mp4')

# prepare video parameters for saving
size = int(scrVideo.get(3)),int(scrVideo.get(4))
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('final_TV.mp4',fourcc, 20, size, True)

# to display video
while scrVideo.isOpened():
    ret, scrFrame = scrVideo.read()
    
    if ret == True:
        # get frame shape
        (hFrame, wFrame) = scrFrame.shape[:2]
        
        # get the masked screen
        maskScr, maskedScreen = find_greenScreen(scrFrame)
        
        # find the rectangular that contains the TV screen 
        pts = get_contours(maskScr)
        
        # read the input video
        ret, inputVideo = InputVideo.read()
        
        # get the shape of the frame
        (hVideo, wVideo) = inputVideo.shape[:2]

        # get frame border to transform it to the TV screen
        ptVideo = np.float32([[0, 0], [wVideo, 0], [wVideo, hVideo], [0, hVideo]])
        # convert TV screen into float
        ptScreen = np.float32(pts)

        # Calculate transformation matrix
        matrix = cv2.getPerspectiveTransform(ptVideo, ptScreen)
        
        # warp input image to the screen
        warpedVideo = cv2.warpPerspective(inputVideo, matrix, (wFrame,hFrame))
        
        # create a mask for the input video, which is the revese of the TV screen mask
        maskVideo = cv2.bitwise_not(maskScr)
        
        # mask input video
        maskedVideo = cv2.bitwise_and(warpedVideo, warpedVideo, mask= maskVideo)
        
        # add two videos together
        output = cv2.add(maskedScreen, maskedVideo)
        
        # save video
        out.write(output)
        # show video
        cv2.imshow('Video', output)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            cv2.destroyWindow('Video')
    
    else:
        break

out.release()  
cv2.destroyWindow('Video')